In [ ]:
# Chapter 1

## 어플리케이션의 3가지 형태

### 싱글 스레드 어플리케이션
- 일반적으로 많이 택하는 형태로 작업 분산은 고려하지 않음

### 멀티 스레드 어플리케이션
- 장착된 CPU가 여러개일 경우 사용하기 좋음
- 어플리케이션이 과부하를 줄 경우 새 스레드나 프로세스를 생성해서 다른 CPU가 분산 처리

### 네트워크 분산 어플리케이션
- 모든 방법을 시도해보고 가장 마지막으로 선택하는 방식

## CPU를 사용한 분산 형태
- 파이썬에서 사용하기엔 큰 약점-GIL이 있음

### GIL (Global Interpreter Lock)
- 파이썬에서 스레드를 여러개로 확장하더라도 이걸로 인해서 성능이 제한적이다,

- 파이썬이 제공하는 일부 객체가 스레드에 대해 안전하다는 걸 보장한다

In [1]:
# 예시 코드
"""
list.append는 원자적연산이다
이게 가능한 이유는 GIL을 통해서 한번에 하나의 스레드만 바이트코드 실행이 가능하기 때문
"""
import threading

x = []

def append_two(l):
    l.append(2)

threading.Thread(target=append_two, args=(x,)).start()
    
x.append(1)
print(x)

[2, 1]


스레드에서 많은 바이트코드를 사용하면 GIL을 얻기위한 경쟁도 발생하여 오히려 싱글스레드보다 안 좋을 수도 있다.
-> 멀티 프로세스를 써보자

## 분산 시스템
- 네트워크를 통해 연결된 서버 여러대를 사용, 즉 어플리케이션은 하나의 노드가 아닌 여러 대의 노드와 연결되어 분산한다.

In [2]:
# Chapter 2 CPU확장

In [9]:
# threading 모듈은 메인 스레드 외의 스레드를 실행하기 위해 제공
# 예제 2.1 스레드생성

import threading

def test_thread():
    def print_something(something):
        print(something)

    t = threading.Thread(target=print_something, args=("hello",))
    t.start()
    print("thread started")
    t.join()

In [12]:
# 순서가 보장되지 않음
for i in range(10):
test_thread()

hello
thread started
hello
thread started
hello
thread started
hello
thread started
hello
thread started
hello
thread started
hello
thread started
hello
thread started
hello
thread started
hello
thread started


In [1]:
"""
프로그램이 스레드의 완료를 기다릴 수 없는 상황이라면 스레드를 데몬으로 만들 수 있다.
이렇게 하면 백그라운드 스레드로 간주되어 메인 스레드가 종료되자마자 같이 종료된다.
"""

#예제 2.2 데몬으로 스레드 생성
import threading

def print_something(something):
    print(something)
    
t = threading.Thread(target=print_something, args=("hello",))
t.daemon = True
t.start()
print("thread started")

hello
thread started


In [3]:
# 예제 2.3 멀티 스레딩을 사용한 워커
import random
import threading

results = []

def compute():
    results.append(sum(
        [random.randint(1, 100) for i in range(1000000)]
    ))

workers = [threading.Thread(target=compute) for x in range(8)]

for worker in workers:
    worker.start()

for worker in workers:
    worker.join()
print("Results: %s" % results)

KeyboardInterrupt: 

## 프로세스 사용하기

In [ ]:
# 예제 2.4 멀티프로세싱 사용하기
import random
import multiprocessing

def compute(results):
    results.append(sum(
        [random.randint(1, 100) for i in range(10000)]
    ))

with multiprocessing.Manager() as manager:
    results = manager.list()
    workers = [multiprocessiong.Process(target=compute, args=(results,)) for x in range(8)]
    
    for worker in workes:
        worker.start()
    for worker in workers:
        worker.join()
    
    print("Results: %s" % results)

In [ ]:
# 2.6 멀티프로세싱을 사용하는 워커
import multiprocessing
import random

pool = multiprocessing.Pool(processes=8)
print("Results: %s" % pool.map(compute, range(8)))